Modelo de Recomendacion Item-Item

importaciones

In [34]:
import pandas as pd 
import numpy as np
import scipy as sp 
from sklearn.metrics.pairwise import cosine_similarity
import operator
import fastparquet as fp 
import pyarrow as pa 
import pyarrow.parquet as pq

In [35]:
df_modelo = pd.read_parquet('../archivos_limpios/games.parquet')
df_modelo

,genres,app_name,url,release_date,reviews_url,price,id,developer
88310,Action,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140.0,Kotoshiro
88310,Casual,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140.0,Kotoshiro
88310,Indie,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140.0,Kotoshiro
88310,Simulation,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140.0,Kotoshiro
88310,Strategy,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140.0,Kotoshiro
...,...,...,...,...,...,...,...,...
120443,Indie,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,http://steamcommunity.com/app/658870/reviews/?...,4.99,658870.0,"xropi,stev3ns"
120444,Adventure,Maze Run VR,http://store.steampowered.com/app/681550/Maze_...,Valor no disponible,http://steamcommunity.com/app/681550/reviews/?...,4.99,681550.0,Valor no disponible
120444,Indie,Maze Run VR,http://store.steampowered.com/app/681550/Maze_...,Valor no disponible,http://steamcommunity.com/app/681550/reviews/?...,4.99,681550.0,Valor no disponible
120444,Action,Maze Run VR,http://store.steampowered.com/app/681550/Maze_...,Valor no disponible,http://steamcommunity.com/app/681550/reviews/?...,4.99,681550.0,Valor no disponible


In [36]:
df_modelo = df_modelo.drop(['url','release_date','reviews_url','price','developer'], axis=1)
df_modelo

,genres,app_name,id
88310,Action,Lost Summoner Kitty,761140.0
88310,Casual,Lost Summoner Kitty,761140.0
88310,Indie,Lost Summoner Kitty,761140.0
88310,Simulation,Lost Summoner Kitty,761140.0
88310,Strategy,Lost Summoner Kitty,761140.0
...,...,...,...
120443,Indie,EXIT 2 - Directions,658870.0
120444,Adventure,Maze Run VR,681550.0
120444,Indie,Maze Run VR,681550.0
120444,Action,Maze Run VR,681550.0


In [37]:
df_modelo['id'].dtype

dtype('float64')

In [38]:
df_modelo['id'] = df_modelo['id'].astype(int)

In [39]:
df_modelo['id'].dtype

dtype('int32')

In [40]:
#Se crean los dummies 
modelo_item = pd.get_dummies(df_modelo, columns=['genres'], prefix='')

modelo_item = modelo_item.groupby(['id', 'app_name']).sum().reset_index()

modelo_item

,id,app_name,_Accounting,_Action,_Adventure,_Animation & Modeling,_Audio Production,_Casual,_Design & Illustration,_Education,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31094,2028055,Tom Clancy's Ghost Recon Future Soldier - Seas...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31095,2028056,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
31096,2028062,Call of Duty®: Black Ops II Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31097,2028103,Assassin’s Creed® III Season Pass,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
similitudes = cosine_similarity(modelo_item.iloc[:,3:])
print(similitudes)

[[1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 ...
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [0.70710678 0.70710678 0.70710678 ... 0.70710678 1.         0.70710678]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]]


In [42]:
similitudes.shape

(31099, 31099)

In [43]:
def recomendacion_juegos(id):
    
    id = int(id)
    # Filtrar el juego e igualarlo a  su ID
    juego_seleccionado = modelo_item[modelo_item['id'] == id]
    # devolver error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo_item[modelo_item['id'] == id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item.iloc[indices_juegos_similares]['app_name']
    
    return juegos_recomendados

In [44]:
recomendacion_juegos(772540)


21000                  Street Champ VR
9969     Star Trek™ : 25th Anniversary
19887                         ARK Park
12251                        Hurtworld
23745                   Country of One
Name: app_name, dtype: object

In [45]:
modelo_item

,id,app_name,_Accounting,_Action,_Adventure,_Animation & Modeling,_Audio Production,_Casual,_Design & Illustration,_Education,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31094,2028055,Tom Clancy's Ghost Recon Future Soldier - Seas...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31095,2028056,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
31096,2028062,Call of Duty®: Black Ops II Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31097,2028103,Assassin’s Creed® III Season Pass,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
#Cuento filas
cant_filas= len(modelo_item)

#Calculo la mitad
mitad_filas= cant_filas // 10
 
#Selecciono la mitad superior
modelo_railway= modelo_item.iloc[:mitad_filas]

In [47]:
modelo_railway.shape


(3109, 22)

In [48]:
similitudes_railway = cosine_similarity(modelo_railway.iloc[:,3:])


In [51]:
modelo_railway.to_parquet('../data/modelo_railway.parquet')


In [54]:
def encontrar_juegos_similares(id):
    # Encuentra el índice del juego ingresado por ID
    juego_indice = modelo_railway.index[modelo_railway['id'] == id].tolist()[0]
    
    # Extrae las características del juego ingresado
    juego_caracteristicas = modelo_railway.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    # Calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_render = cosine_similarity(modelo_railway.iloc[:, 3:], juego_caracteristicas)
    
    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_render.argsort(axis=0)[::-1][1:6]
    indices_juegos_similares = indices_juegos_similares.flatten()[1:]
    
    # Obtiene los juegos más similares en función de los índices
    juegos_similares = modelo_railway.iloc[indices_juegos_similares]['app_name']
    
    return juegos_similares

In [55]:
encontrar_juegos_similares(3310)


908                      Flora's Fruit Farm
886                            Columns™ III
1219                         Zombie Pirates
452     Samantha Swift and the Golden Touch
Name: app_name, dtype: object

In [56]:
def recomendacion_juego(id: int):
    
    # Verificamos si el juego con game_id existe en df_games
    game = modelo_railway[modelo_railway['id'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # Obtenemos el índice del juego dado
    idx = game.index[0]

    # Tomamos una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Definimos el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_railway.sample(n=sample_size, random_state=42)  # Ajustamos la semilla aleatoria según sea necesario

    # Calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_railway.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtenemos los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Listamos los juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}


In [57]:
recomendacion_juego(3310)

{'similar_games': ['SlamIt Pinball Big Score',
  'Samantha Swift and the Hidden Roses of Athena',
  'Zuma Deluxe',
  'Mystery P.I.™ - The Vegas Heist',
  'YOU DON’T KNOW JACK®']}